In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [4]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA2a/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [5]:
mineralogy = mineralogy.dropna()

In [6]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [7]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [8]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [9]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
12278,70.776896,0.363839,15.301404,0.810033,2.540332,0.057361,0.597421,1.768580,3.573705,4.210429
12279,70.755807,0.364824,15.308361,0.814077,2.544270,0.057454,0.600199,1.775138,3.573614,4.206256
12280,70.733481,0.365861,15.315712,0.818135,2.548568,0.057555,0.603182,1.782144,3.573516,4.201847
12281,70.709910,0.366948,15.323457,0.822201,2.553233,0.057663,0.606374,1.789603,3.573408,4.197202
12282,70.685088,0.368086,15.331595,0.826268,2.558269,0.057779,0.609779,1.797523,3.573290,4.192323


In [10]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
12278,70.776896,0.363839,15.301404,0.810033,2.540332,0.057361,0.597421,1.768580,3.573705,4.210429
12279,70.755807,0.364824,15.308361,0.814077,2.544270,0.057454,0.600199,1.775138,3.573614,4.206256
12280,70.733481,0.365861,15.315712,0.818135,2.548568,0.057555,0.603182,1.782144,3.573516,4.201847
12281,70.709910,0.366948,15.323457,0.822201,2.553233,0.057663,0.606374,1.789603,3.573408,4.197202
12282,70.685088,0.368086,15.331595,0.826268,2.558269,0.057779,0.609779,1.797523,3.573290,4.192323
...,...,...,...,...,...,...,...,...,...,...
209133,73.028858,0.246689,14.289525,1.395818,1.627995,0.055340,0.337313,0.887992,3.650248,4.480221
209134,73.017473,0.247133,14.295364,1.396654,1.630897,0.055435,0.338743,0.890747,3.650436,4.477117
209160,71.290260,0.319028,15.026361,1.607536,2.034157,0.065490,0.555807,1.340252,3.648979,4.112129
209161,71.261899,0.320309,15.036440,1.611503,2.040510,0.065586,0.559309,1.348382,3.648535,4.107526


In [11]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]

In [12]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA2a/area2a_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA2a/area2a_2.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA6/area6_3.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA6/area6_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [16]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2a/area2a_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA2a/area2a_2_results_excel.txt")
#df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_3_results_excel.txt")
#df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_4_results_excel.txt")

In [17]:
df_final = pd.concat([df_final1, df_final2])

In [18]:
df_final

,Q,P,K
SAMPLE,,,
12278,27.598455,35.987262,27.903727
12279,27.574537,36.019120,27.879010
12280,27.548782,36.053158,27.852874
12281,27.521280,36.089328,27.825303
12282,27.491844,36.127705,27.796365
...,...,...,...
209133,31.544332,32.201784,29.562720
209134,31.533775,32.216924,29.544563
209160,29.712883,34.435673,27.386555


## QAPF classification

In [19]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [20]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [21]:
df_final.to_excel("../_CIPW/CIPW/AREA2a/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA2a/QAPF_counts.xlsx")

-----

In [24]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area2_a.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [26]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
47,755477.668714,5.262227e+06,53.0,T,6.0,41.044247,25.818789,29.731042,monzo granite
66,718647.819565,5.235770e+06,53.0,T,6.0,39.887051,27.689473,28.221978,monzo granite
86,731796.073283,5.259752e+06,53.0,T,6.0,37.527641,27.835529,31.300823,monzo granite
152,755477.629709,5.262227e+06,53.0,T,6.0,44.170254,24.159087,26.096767,monzo granite
161,734741.917204,5.301092e+06,53.0,T,6.0,40.339199,24.016544,30.965927,monzo granite
...,...,...,...,...,...,...,...,...,...
4587,671281.488145,5.177993e+06,53.0,T,6.0,0.000000,42.779899,29.170267,monzonite
4595,735289.609871,5.294808e+06,53.0,T,6.0,7.144572,53.407395,17.261420,quartz monzodiorite\nquartz monzogabbro
4597,670787.218612,5.216880e+06,53.0,T,6.0,7.045293,52.205423,13.724197,quartz monzodiorite\nquartz monzogabbro
4638,746609.482852,5.293856e+06,53.0,T,6.0,6.645143,62.556135,9.683687,quartz monzodiorite\nquartz monzogabbro


In [28]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA2a/QAPF_control_points.xlsx", index=True)